# Nighttime Lights Trends in Gaza and West Bank

Analyzing conflict dynamics through the lens of [NASA's Black Marble Nighttime Lights](https://blackmarble.gsfc.nasa.gov) dataset opens a unique window into the often-hidden facets of global unrest. In a world marked by diverse forms of conflict, from armed confrontations to civil unrest, the dataset offers an unconventional yet powerful tool for understanding the ripple effects of these conflicts on human settlements and infrastructure. By tracking nighttime light variations and disruptions, we can unearth vital insights into population displacement, economic destabilization, and the societal impacts of conflict. This analysis explores the potential of the Black Marble Nighttime Lights dataset to not only detect areas affected by conflict but also to quantify the extent of its influence on human lives and livelihoods, providing a valuable perspective on the multifaceted consequences of conflict worldwide.

In [1]:
import math
import os
from datetime import datetime
from math import pi

import colorcet as cc
import dask.dataframe as dd
import folium
import geopandas
import numpy as np
import pandas as pd
from blackmarble.extract import bm_extract
from blackmarble.raster import bm_raster
from bokeh.layouts import column as BOKEH_COLUMN
from bokeh.layouts import gridplot
from bokeh.models import (
    BasicTicker,
    ColumnDataSource,
    Div,
    HoverTool,
    Legend,
    PrintfTickFormatter,
    Range1d,
    Span,
    TabPanel,
    Tabs,
    Text,
    Title,
)
from bokeh.plotting import figure, output_notebook, show
from bokeh.transform import linear_cmap

In [2]:
# https://papermill.readthedocs.io/en/latest/usage-parameterize.html
VAR = "ntl_mean"

## Data

### Define Region of Interest

Define region of interest for where NASA Black Marble will be downloaded.

In [3]:
PSE = geopandas.read_file("../../data/boundaries/gadm41_PSE_shp/gadm41_PSE_2.shp")

PSE.explore()

```{figure} ../../docs/images/logo.png
---
height: 0px
---
Region of Interest. Country borders or names do not necessarily reflect the World Bank Group's official position. This map is for illustrative purposes and does not imply the expression of any opinion on the part of the World Bank, conceming the legal status of any country or territory or concerning the delimitation of frontiers or boundaries. 
```

### Black Marble 

[NASA's Black Marble](https://blackmarble.gsfc.nasa.gov) VIIRS (Visible Infrared Imaging Radiometer Suite) Nighttime Lights product suite represents a remarkable advancement in our ability to monitor and understand nocturnal light emissions on a global scale. By utilizing cutting-edge satellite technology and image processing techniques, the Black Marble VIIRS dataset offers a comprehensive and high-resolution view of the Earth's nighttime illumination patterns. 

In [ ]:
VNP46A2 = bm_extract(
    PSE,
    product_id="VNP46A2",
    date_range=pd.date_range("2022-01-01", "2023-12-31", freq="D"),
    bearer=os.environ.get("BLACKMARBLE_TOKEN"),
    aggfunc=["mean", "sum"],
)

In [ ]:
VNP46A3 = bm_extract(
    PSE,
    product_id="VNP46A3",
    date_range=pd.date_range("2022-01-01", "2023-12-31", freq="MS"),
    bearer=os.environ.get("BLACKMARBLE_TOKEN"),
    aggfunc=["mean", "sum"],
)

In [6]:
VNP46A3 = (
    VNP46A3.pivot_table(index="date", columns=["NAME_2"], values=[VAR], aggfunc="mean")
    .resample("MS", label="left")
    .mean()
)

In [7]:
VNP46A2[VAR] *= 0.1
VNP46A3[VAR] *= 0.1

The latest update date is:

In [8]:
VNP46A2["date"].max().strftime("%d %B %Y (Week %W)")

'08 December 2023 (Week 49)'

```{important}
The [VNP46A2 Daily Moonlight-adjusted Nighttime Lights (NTL) Product](https://blackmarble.gsfc.nasa.gov/VNP46A2.html) is [available daily](datahttps://ladsweb.modaps.eosdis.nasa.gov/missions-and-measurements/products/VNP46A2/#data-availability). However, due data quality, cloud cover or other factors, the data may not be available always at a specific location.
```

## Methodology

Creating a time series of weekly radiance using NASA's Black Marble data involves several steps, including data acquisition, pre-processing, zonal statistics calculation, and time series generation. Below is a general methodology for this process.

###  Time Series Generation

Organize the zonal statistics results in a tabular format, where each columnn corresponds to a specific zone, and rows represent the daily radiance values. Next, we aggregate the data on a weekly basis, computing the desired statistical metric (e.g., mean radiance) for each zone for each week. Finally, we will visualize the time series data to observe trends, patterns, and anomalies over time.

#### Weekly

In this step, we compute a weekly aggregation of the zonal statistics by for each second-level administrative division and for each week. In this case, we `W-SUN` and `mean` as aggregate function.

In [9]:
PSE_2 = (
    VNP46A2.pivot_table(values=[VAR], index="date", columns=["NAME_2"])
    .resample("W-SUN", label="right")
    .mean()
)
PSE_1 = (
    VNP46A2.pivot_table(values=[VAR], index="date", columns=["NAME_1"])
    .resample("W-SUN", label="right")
    .mean()
)

PSE_2

ntl_mean                                                          \
NAME_2      Bethlehem Deir Al-Balah       Gaza Gaza ash Shamaliyah     Hebron   
date                                                                            
2022-01-02  15.204650      7.979566  13.673667            8.275610   5.509554   
2022-01-09   3.937239     13.335583  20.106966           12.400183   6.795771   
2022-01-16   9.067182     11.666783  15.247343           11.185364   4.952166   
2022-01-23  11.885563     13.398174  19.648510           13.627723   6.976829   
2022-01-30   3.984767     10.135582  14.438217            9.784870   4.517948   
...               ...           ...        ...                 ...        ...   
2023-11-12   7.756458      1.507488   2.757916            2.406718  10.918434   
2023-11-19   6.550502      1.168209   1.502485            3.581889   8.362013   
2023-11-26   7.156228      1.333906   1.696037            4.665675   9.008952   
2023-12-03   7.759852      0.975095   2.132110            7.532296   8.911090   
2023-12-10   5.470043      1.038473   1.294303            2.900198   8.252836   

                                                                             \
NAME_2          Jenin   Jericho  Jerusalem Khan Yunis     Nablus   Qalqilya   
date                                                                          
2022-01-02   4.963882  5.526086  26.562299  12.340092   7.097979  11.070501   
2022-01-09   7.521570  5.598589  21.751601  13.603669   6.986950  17.083686   
2022-01-16   6.438554  4.494609  29.346998  13.005158   7.004429  14.890246   
2022-01-23   7.256974  4.372017  22.756302  12.409488   7.997001  17.848442   
2022-01-30   4.674371  4.451629  18.479181  10.809777   4.882015  12.376466   
...               ...       ...        ...        ...        ...        ...   
2023-11-12  10.096802  6.133832  22.052848   1.918821   9.557866  15.885003   
2023-11-19   8.276276  5.835015  26.971692   1.437901   7.638950  21.120911   
2023-11-26   9.448789  6.429732  22.487371   1.634249  10.161027  18.863102   
2023-12-03   9.155024  5.986104  22.749573   1.302449   8.406441  19.816523   
2023-12-10   8.238544  5.389962  18.669865   1.528795   4.526730  15.682873   

                                                                             
NAME_2          Rafah Ramallah and Al-Bireh     Salfit     Tubas    Tulkarm  
date                                                                         
2022-01-02        NaN              9.829518  14.304472  3.700941   5.659215  
2022-01-09  22.926138              9.664528  15.148751  2.439910  11.244419  
2022-01-16  22.234868              9.463288  12.201209  1.950268  11.332975  
2022-01-23  20.634801              8.747132  13.980935  2.780365  11.759398  
2022-01-30  16.943309              6.506664  10.623585  2.216819   8.362207  
...               ...                   ...        ...       ...        ...  
2023-11-12   8.344134             10.463186  18.584041  4.042679  12.660648  
2023-11-19   4.689249             11.773393  17.621894  3.637760  15.383386  
2023-11-26   5.144630             11.334735  16.124129  3.920568  12.576395  
2023-12-03   5.199526              9.229725  19.768423  3.597289  12.000036  
2023-12-10   5.368840              8.418583  14.194544  2.411772   8.457151  

[102 rows x 16 columns]

#### Monthly

In this step, we compute a monthy aggregation of the zonal statistics by for each second-level administrative division and for each month. Additionaly, we add the [VNP46A3](https://ladsweb.modaps.eosdis.nasa.gov/missions-and-measurements/products/VNP46A3/#data-availability) monthly composite, when available.

In [10]:
PSE_2_MS = (
    VNP46A2.pivot_table(index="date", columns=["NAME_2"], values=[VAR], aggfunc="mean")
    .resample("MS", label="left")
    .mean()
)
PSE_2_MS.to_csv("../../data/ntl/PSE_2-VNP46A2-M-zs.csv")

In [11]:
p = figure(
    title="Palestine: Monthly Nighttime Lights (2022-2023)",
    width=800,
    height=600,
    x_axis_label="Date",
    x_axis_type="datetime",
    y_axis_label=r"Radiance [nW $$cm^{-2}$$ $$sr^{-1}$$]",
    tools="pan,wheel_zoom,box_zoom,reset,save,box_select",
)
p.add_layout(
    Title(
        text=f"Monthly NTL Radiance Average (VNP46A2 and VNP46A3) for each second-level administrative division",
        text_font_size="12pt",
        text_font_style="italic",
    ),
    "above",
)
p.add_layout(
    Title(
        text=f"Data Source: NASA Black Marble. Creation date: {datetime.today().strftime('%d %B %Y')}. Feedback: datalab@worldbank.org.",
        text_font_size="10pt",
        text_font_style="italic",
    ),
    "below",
)
p.add_layout(Legend(), "right")

p.add_tools(
    HoverTool(
        tooltips=[
            ("Month", "@x{%B %Y}"),
            ("Radiance", "@y{0.00}"),
        ],
        formatters={"@x": "datetime"},
    )
)
renderers = []
for column, color in zip(PSE_2_MS.columns, cc.b_glasbey_category10):
    try:
        r = p.line(
            PSE_2_MS.index,
            PSE_2_MS[column],
            legend_label=column[1],
            line_color=color,
            line_width=2,
        )
        r.visible = False
        renderers.append(r)
    except:
        pass
renderers[-1].visible = True

for column, color in zip(VNP46A3.columns, cc.b_cyclic_grey_15_85_c0):
    r = p.line(
        VNP46A3.index,
        VNP46A3[column],
        legend_label=column[1],
        line_color=color,
        line_width=2,
    )
    r.visible = False
    renderers.append(r)
renderers[-1].visible = True

p.legend.location = "bottom_left"
p.legend.click_policy = "hide"
p.title.text_font_size = "16pt"

output_notebook()
show(p)

Loading BokehJS ...

```{figure} ../../docs/images/logo.png
---
height: 0px
---
Monthly NTL radiance average derived from the VNP46A2 daily composite and, in grey, derived from the VNP46A3 monthly composite.
```

## Findings

### Percent Change in NTL Radiance 

#### Benchmark Comparison

In this exploratory analysis, we conducted analysis of NTL radiance trends, comparing the observed average radiance levels to a benchmark established in the year 2022 for each second-level administrative division.

In [12]:
PERCENTCHANGE_2 = 100 * (
    PSE_2 / PSE_2[(PSE_2.index >= "2022-01-01") & (PSE_2.index < "2023-01-01")].mean()
    - 1
)
PERCENTCHANGE_1 = 100 * (
    PSE_1 / PSE_1[(PSE_1.index >= "2022-01-01") & (PSE_2.index < "2023-01-01")].mean()
    - 1
)
pd.set_option("display.max_rows", None)
PERCENTCHANGE_2[PERCENTCHANGE_2.index >= "2023-10-01"].style.map(
    lambda x: "background-color: #DF4661" if x < -50 else "background-color: white"
)

In [13]:
def create_plot(data):
    p = figure(
        title="Palestine: Percent Change in Nighttime Lights Radiance Average",
        width=800,
        height=600,
        x_axis_label="Date",
        x_axis_type="datetime",
        y_axis_label="NTL Percent Change (%)",
        tools="pan,wheel_zoom,box_zoom,reset,save,box_select",
    )
    # p.y_range = Range1d(-100, 50, bounds=(-100, 100))
    p.xaxis.major_label_orientation = math.pi / 4
    p.add_layout(
        Title(
            text=f"Weekly percent change (2022 baseline) in NTL radiance for each second-level administrative division",
            text_font_size="12pt",
            text_font_style="italic",
        ),
        "above",
    )
    p.add_layout(
        Title(
            text=f"Source: NASA Black Marble. Creation date: {datetime.today().strftime('%d %B %Y')}. Feedback: datalab@worldbank.org.",
            text_font_size="10pt",
            text_font_style="italic",
        ),
        "below",
    )
    p.add_layout(Legend(), "right")
    p.renderers.extend(
        [
            Span(
                location=datetime(2023, 10, 7),
                dimension="height",
                line_color="gray",
                line_width=1.5,
                line_dash=(4, 4),
            ),
        ]
    )
    p.add_tools(
        HoverTool(
            tooltips=[
                ("Date", "@x{%F} (@x{%W of %Y})"),
                ("Percent Change", "@y{0.0}%"),
            ],
            formatters={"@x": "datetime"},
        )
    )
    renderers = []
    for column, color in zip(data.columns, cc.b_glasbey_category10):
        r = p.line(
            data.index,
            data[column],
            legend_label=str(column[1]),
            line_color=color,
            line_width=2,
        )
        r.visible = False
        renderers.append(r)
    renderers[2].visible = True

    p.legend.location = "bottom_left"
    p.legend.click_policy = "hide"
    p.title.text_font_size = "16pt"
    # p.sizing_mode = "scale_both"
    return p

In [14]:
PERCENTCHANGE_2.to_csv("../../data/ntl/PSE_2-VNP46A2-WSUN-percentchange.csv")

In [15]:
show(create_plot(PERCENTCHANGE_2))

```{figure} ../../docs/images/logo.png
---
height: 0px
---
Weekly percent change (compared to 2022 benchmark) in NTL radiance (VNP46A2) for each second-level administrative division. 
```

Additionally, we visualize below the NTL radiance average on first-level administrative division. 

In [16]:
def create_plot_individual(data, colors):
    p = figure(
        title=data.columns[0][1],
        width=800,
        height=600,
        x_axis_label="Date",
        x_axis_type="datetime",
        y_axis_label="NTL Percent Change (%)",
        tools="pan,wheel_zoom,box_zoom,reset,save,box_select",
    )
    p.y_range = Range1d(-100, 40, bounds=(-100, 100))
    p.xaxis.major_label_orientation = math.pi / 4
    p.add_layout(Legend(), "right")
    p.renderers.extend(
        [
            Span(
                location=datetime(2023, 10, 7),
                dimension="height",
                line_color="red",
                line_width=1.5,
                line_dash=(4, 4),
            ),
        ]
    )
    p.add_tools(
        HoverTool(
            tooltips=[
                ("Date", "@x{%F} (@x{%W of %Y})"),
                ("Percent Change", "@y{0.0}% (2022 baseline)"),
            ],
            formatters={"@x": "datetime"},
            mode="vline",
        )
    )
    p.add_layout(
        Title(
            text=f"Weekly percent change (2022 baseline) in NTL radiance",
            text_font_size="12pt",
            text_font_style="italic",
        ),
        "above",
    )
    renderers = []
    for column, color in zip(data.columns, colors):
        r = p.line(
            data.index,
            data[column],
            line_color=color,
            line_width=2,
        )
        renderers.append(r)

    p.legend.location = "bottom_left"
    p.legend.click_policy = "hide"
    p.title.text_font_size = "24pt"
    # p.sizing_mode = "scale_both"

    return p

In [17]:
for column, color in zip(PERCENTCHANGE_1.columns, cc.bmy):
    p = create_plot_individual(PERCENTCHANGE_1[column].to_frame(), [color])
    show(p)

# for column, color in zip(PERCENTCHANGE_2.columns, cc.b_glasbey_category10):
#     p = create_plot_individual(PERCENTCHANGE_2[column].to_frame(), [color])

```{figure} ../../docs/images/logo.png
---
height: 0px
---
Weekly percent change (compared to 2022 benchmark) in NTL radiance (NASA Black Marble VNP46A2) for each first-level administrative division. 
```

#### Week over Week Comparison

In this exploratory analysis, we conducted analysis of NTL radiance trends, comparing the observed average radiance levels week over week (WOW) for each second-level administrative division.

In [18]:
WOW_2 = 100 * (PSE_2.pct_change(1, axis="rows", fill_method=None))

In [19]:
WOW_2.to_csv("../../data/ntl/PSE_2-VNP46A2-WOW-percentchange.csv")

In [20]:
p = figure(
    title="Palestine: Percent Change in Nighttime Lights Radiance",
    width=800,
    height=600,
    x_axis_label="Date",
    x_axis_type="datetime",
    y_axis_label="NTL Percent Change (%)",
    tools="pan,wheel_zoom,box_zoom,reset,save,box_select",
)
p.xaxis.major_label_orientation = math.pi / 4
p.add_layout(
    Title(
        text=f"Weekly percent change week over week in NTL radiance for each second-level administrative division",
        text_font_size="12pt",
        text_font_style="italic",
    ),
    "above",
)
p.add_layout(
    Title(
        text=f"Source: NASA Black Marble. Creation date: {datetime.today().strftime('%d %B %Y')}. Feedback: datalab@worldbank.org.",
        text_font_size="10pt",
        text_font_style="italic",
    ),
    "below",
)
p.add_layout(Legend(), "right")
p.renderers.extend(
    [
        Span(
            location=datetime(2023, 10, 7),
            dimension="height",
            line_color="gray",
            line_width=1.5,
            line_dash=(4, 4),
        ),
    ]
)
p.add_tools(
    HoverTool(
        tooltips=[
            ("Week", "@x{%W} (@x{%F})"),
            ("Percent Change", "@y{0.00}% (week over week)"),
        ],
        formatters={"@x": "datetime"},
    )
)
renderers = []
for column, color in zip(WOW_2.columns, cc.b_glasbey_category10):
    r = p.line(
        WOW_2.index,
        WOW_2[column],
        legend_label=str(column[1]),
        line_color=color,
        line_width=2,
    )
    r.visible = False
    renderers.append(r)

renderers[2].visible = True

p.legend.location = "bottom_left"
p.legend.click_policy = "hide"
p.title.text_font_size = "16pt"
# p.sizing_mode = "scale_both"

show(p)

### Point-in-Time Comparison

#### Daily

```{figure} ./figures/pse_ntl_VNP46A1_2023-01-01.png
---
height: 400px
---
Nighttime lights on January 1, 2023. Source: NASA Black Marble (VNP46A1).
```

```{figure} ./figures/pse_ntl_VNP46A1_2023-10-21.png
---
height: 400px
---
Nighttime lights on October 21, 2023. Source: NASA Black Marble (VNP46A1).
```

#### Weekly
We visualize below weekly snapshots of the percent change (compared to 2022) in NTL radiance average for each second-level administrative division. 

In [21]:
group = lambda flat, size: [flat[i : i + size] for i in range(0, len(flat), size)]

In [22]:
m = PSE.merge(
    PERCENTCHANGE_2.iloc[1]
    .to_frame("percent change in radiance average (%)")
    .reset_index(),
    on="NAME_2",
).explore(
    column="percent change in radiance average (%)", cmap="cividis", vmin=-100, vmax=0
)

title_html = f"""<h3 align="center" style="font-size:20px">Week of <b>{PERCENTCHANGE_2.iloc[0].name.strftime("%F")}</b></h3>"""
m.get_root().html.add_child(folium.Element(title_html))
m

In [23]:
m = PSE.merge(
    PERCENTCHANGE_2.loc["2023-10-08"]
    .to_frame("percent change in radiance average (%)")
    .reset_index(),
    on="NAME_2",
).explore(
    column="percent change in radiance average (%)", cmap="cividis", vmin=-100, vmax=0
)

title_html = f"""<h3 align="center" style="font-size:20px">Week of <b>{PERCENTCHANGE_2.loc["2023-10-08"].name.strftime("%F")}</b></h3>"""
m.get_root().html.add_child(folium.Element(title_html))
m

In [24]:
m = PSE.merge(
    PERCENTCHANGE_2.iloc[-1]
    .to_frame("percent change in radiance average (%)")
    .reset_index(),
    on="NAME_2",
).explore(
    column="percent change in radiance average (%)", cmap="cividis", vmin=-100, vmax=0
)

title_html = f"""<h3 align="center" style="font-size:20px">Week of <b>{PERCENTCHANGE_2.iloc[-1].name.strftime("%F")}</b></h3>"""
m.get_root().html.add_child(folium.Element(title_html))
m

## Limitations 

Using nighttime lights to estimate macroeconomic indicators during conflict may be a valuable approach, but it comes with several assumptions and limitations. Here's a list of some of the key assumptions and limitations:

```{caution}
**Assumptions:**

- **Luminosity Reflects Economic Activity:** The approach assumes that the level of nighttime lights is a reliable proxy for economic activity. It presupposes that areas with brighter lights correspond to higher economic productivity.

- **Baseline Data Availability:** It assumes the availability of baseline nighttime lights data before the onset of the conflict. The accuracy of the estimates depends on the quality and relevance of this baseline data.

- **Spatial Distribution:** The method assumes that nighttime lights are evenly distributed within a given geographic area and that changes in luminosity accurately reflect changes in economic activity across all locations.
    

**Limitations:**

- **Confounding Factors and Data Interpretation:** The approach may require subjective interpretation, as it may not distinguish between reduced lighting due to conflict and reduced lighting due to other factors. Changes in nighttime lights can be influenced by factors other than economic activity, such as energy conservation measures, urban development, or seasonal variations.

- **Generalization:** The approach might lead to overgeneralization, as a reduction in nighttime lights can be associated with various economic outcomes, from minor disruptions to severe economic downturns.

- **Alternative Explanations:** Changes in nighttime lights can result from factors other than conflict, such as urban development, changes in economic activities, or natural disasters. Therefore, it may not always be clear whether a decline in nighttime lights is solely due to conflict.

- **Geopolitical Factors:** The dataset may be subject to geopolitical biases, with some areas having less comprehensive coverage due to political reasons.

- **Data Lag:** There can be a significant time lag between the occurrence of a conflict event and its reflection in the nighttime lights dataset. This lag may limit the dataset's utility for real-time conflict monitoring.

- **Resolution and Urban Bias:** The dataset's spatial resolution may not be fine enough to capture small villages or isolated conflict events. It may also have an urban bias, making it less suitable for analyzing rural or remote conflicts.
```

To address these assumptions and limitations, it is crucial to complement nighttime lights data analysis with other sources of information and adopt a cautious and context-aware approach when interpreting the findings.

## References

{cite:empty}`ROMAN2018113`

```{bibliography}
:filter: docname in docnames
:style: plain
```